In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, ks_2samp, f_oneway
from sklearn.preprocessing import StandardScaler

from utils2_check import *

pd.set_option('future.no_silent_downcasting', True)
pd.set_option('display.max_columns', None)

In [5]:
# import importlib
# importlib.reload(stat_utils_check)
# from utils import make_graph
from stat_utils_check import *

In [7]:
path = "C:/Users/82104/Downloads/Insurance_fraud/"
df = pd.read_csv(path + "fraud_oracle.csv")

In [8]:
time_vars = ["Month", "WeekOfMonth", "DayOfWeek", "DayOfWeekClaimed", 'MonthClaimed', 'WeekOfMonthClaimed']
vehicle_vars = ["Make", "VehiclePrice", "VehicleCategory", "AgeOfVehicle"]
personal_vars = ["Sex", 'MaritalStatus', "Age", 'DriverRating', 'AgeOfPolicyHolder', 'NumberOfCars', 'PastNumberOfClaims']
policy_vars = ["PolicyType", 'Deductible', 'AgentType', "NumberOfSuppliments", "BasePolicy"]
accident_vars = ['Days_Policy_Accident', 'Days_Policy_Claim', 'PoliceReportFiled', 'WitnessPresent', 'AddressChange_Claim']

1. policy type

In [9]:
# 1의 Liability가 너무작다. 조금 제쳐두자
plot_count_plot(df, "BasePolicy", "FraudFound_P")

FraudFound_P


In [20]:
collision = df.loc[df['BasePolicy']=="Collision"]
liability = df.loc[df['BasePolicy']=="Liability"]
all_p = df.loc[df['BasePolicy']=="All Perils"]

In [21]:
collision.shape

(5962, 33)

In [22]:
collision.FraudFound_P.value_counts()

FraudFound_P
0    5527
1     435
Name: count, dtype: int64

In [23]:
435/5962

0.0729620932572962

In [24]:
liability.shape, liability.FraudFound_P.value_counts()

((5009, 33),
 FraudFound_P
 0    4973
 1      36
 Name: count, dtype: int64)

In [25]:
36/5009

0.007187063286085047

In [26]:
all_p.shape, all_p.FraudFound_P.value_counts()

((4449, 33),
 FraudFound_P
 0    3997
 1     452
 Name: count, dtype: int64)

In [27]:
452/4449

0.10159586423915487

In [28]:
output, rel_cols_all = statistical_tests_step1(all_p, "FraudFound_P", [i for i in all_p.columns if i!='FraudFound_P'])

Relevant : AccidentArea
Relevant : MonthClaimed
Relevant : Age
Relevant : Fault
Relevant : VehiclePrice
Relevant : PolicyNumber
Relevant : AgeOfVehicle
Relevant : AgeOfPolicyHolder
Relevant : PoliceReportFiled
Relevant : NumberOfSuppliments
Relevant : AddressChange_Claim
Relevant : Year


In [29]:
output, rel_cols_collision = statistical_tests_step1(collision, "FraudFound_P", [i for i in collision.columns if i!='FraudFound_P'])

Relevant : Make
Relevant : MonthClaimed
Relevant : Sex
Relevant : Fault
Relevant : PolicyType
Relevant : VehicleCategory
Relevant : VehiclePrice
Relevant : AddressChange_Claim


In [30]:
# 너무 빈도가 낮기때문에 무시
# output, rel_cols_liability = statistical_tests_step1(liability, "FraudFound_P", [i for i in collision.columns if i!='FraudFound_P'])

In [31]:
# 단순히 가격이 관련있을것같음
[i for i in rel_cols_all if i in rel_cols_collision]

['MonthClaimed', 'Fault', 'VehiclePrice', 'AddressChange_Claim']

---

# 가격이 높은 차량에 대한 보험 사기가 높을 것이다.
- (구체화) Collision에서 이런 현상이 더욱 더 두드러지게 나타날 것이다
- (구체화) 게다가 과거 사고 이력도 많을 것이다
- (비즈니스 근거) 자동차의 가격이 높을수록 더 많은 보험금이 지급되며, 과거 보험금을 몇 번 수령하였을 것이다 (이때, 걸리지 않고)
- (결론 및 활용) 위 항목들을 기준으로 보험 사기 검사를 통해 사기를 더 많이 잡아낼 수 있을 것이다.

In [32]:
plot_count_plot(df, 'VehiclePrice')

In [33]:
# 충돌나면 상대방 차, 물건에 관해서만 보험 적용.
# 커버범위, 보험금 작음
# 따라서 저렴한 편의 차가 주로 가입함

plot_count_plot(collision, 'VehiclePrice')

In [34]:
# 종합보험은 더 많은 커버리지를 가져감.
# 비싼 차는 사고나면 더 많은 수리비, 보험금 필요.
# 따라서 종합상품으로 가격을 디펜스함. 
plot_count_plot(all_p, 'VehiclePrice')

In [35]:
plot_two_categorical(df, 'VehiclePrice', 'FraudFound_P', 'heatmap')

In [36]:
# 자동차 가격에 따라 비율이 달라짐을 확인 가능
plot_two_categorical(collision, 'VehiclePrice', 'FraudFound_P', 'heatmap')

In [37]:
plot_count_plot(collision, 'VehiclePrice', 'FraudFound_P')

FraudFound_P


In [38]:
plot_two_categorical(all_p, 'VehiclePrice', 'FraudFound_P', 'heatmap')

In [39]:
plot_count_plot(collision, 'VehiclePrice', 'FraudFound_P')

FraudFound_P


In [40]:
plot_count_plot(all_p, 'VehiclePrice', 'FraudFound_P')

FraudFound_P


In [41]:
# 보기좋게 칼럼 합침
vehicle_price_mapping = {
    'less than 20000': "less than 30000",
    '20000 to 29000': "less than 30000",
    '30000 to 39000': "30000 to 59000",
    '40000 to 59000': "30000 to 59000",
    '60000 to 69000': "more than 60000",
    'more than 69000': "more than 60000"
}

# Apply the mapping
df['VehiclePrice_num'] = df['VehiclePrice'].map(vehicle_price_mapping)

collision = df.loc[df['BasePolicy']=="Collision"]
laibility = df.loc[df['BasePolicy']=="Liability"]
all_p = df.loc[df['BasePolicy']=="All Perils"]

In [43]:
df.VehiclePrice_num.value_counts()

VehiclePrice_num
less than 30000    9175
30000 to 59000     3994
more than 60000    2251
Name: count, dtype: int64

In [45]:
2251/(2251+3994+9175)

0.1459792477302205

In [42]:
plot_count_plot(all_p, 'VehiclePrice_num', 'FraudFound_P')

FraudFound_P


In [23]:
statistical_tests_step1(collision, 'FraudFound_P', ['VehiclePrice_num'])

Relevant : VehiclePrice_num


({'VehiclePrice_num': {'chi2': 20.708845507266407,
   'p': 3.185160599115437e-05,
   'dof': 2,
   'expected': array([[1435.98171754,  113.01828246],
          [3356.80426032,  264.19573968],
          [ 734.21402214,   57.78597786]]),
   'contingency_table': FraudFound_P         0    1
   VehiclePrice_num           
   30000 to 59000    1423  126
   less than 30000   3396  225
   more than 60000    708   84}},
 ['VehiclePrice_num'])

In [47]:
statistical_tests_step1(all_p, 'FraudFound_P', ['VehiclePrice_num'])

Relevant : VehiclePrice_num


({'VehiclePrice_num': {'chi2': 8.616635596650708,
   'p': 0.01345616655167756,
   'dof': 2,
   'expected': array([[ 703.4504383 ,   79.5495617 ],
          [2546.07732075,  287.92267925],
          [ 747.47224095,   84.52775905]]),
   'contingency_table': FraudFound_P         0    1
   VehiclePrice_num           
   30000 to 59000     718   65
   less than 30000   2552  282
   more than 60000    727  105}},
 ['VehiclePrice_num'])

In [48]:
plot_two_categorical(all_p, 'VehiclePrice_num', 'FraudFound_P', 'heatmap')

In [49]:
all_p.FraudFound_P.value_counts()

FraudFound_P
0    3997
1     452
Name: count, dtype: int64

In [21]:
2552/3997, 282/452 # 저렴한 차량의 비율

(0.6384788591443583, 0.6238938053097345)

In [30]:
# 비싼차량일수록 보험사기 비율이 높다
727/3997, 105/452 # 비싼 차량의 비율

(0.18188641481110834, 0.2323008849557522)

In [31]:
718/3997, 65/452 # 평균 차량의 비율

(0.1796347260445334, 0.14380530973451328)

In [46]:
plot_two_categorical(collision, 'VehiclePrice_num', 'FraudFound_P', 'heatmap')

In [53]:
225/(3396+225)

0.06213753106876554

In [33]:
collision.FraudFound_P.value_counts()

FraudFound_P
0    5527
1     435
Name: count, dtype: int64

In [34]:
3396/5527, 225/435 # 저가 차량

(0.6144382124117966, 0.5172413793103449)

In [35]:
708/5527, 84/435 # 비싼 차량의 비율

(0.12809842590917314, 0.19310344827586207)

In [36]:
1423/5527, 126/435 # 평균 차량의 비율

(0.2574633616790302, 0.2896551724137931)

In [37]:
# 2개의 컬럼 비교해줌
# 충돌상품
# 3~4퍼 선호함 평균 가격에선
# 7퍼 가까이 차이남 비싼차량
pd.crosstab(collision.VehiclePrice_num, collision.FraudFound_P, normalize='columns')

FraudFound_P,0,1
VehiclePrice_num,,
30000 to 59000,0.257463,0.289655
less than 30000,0.614438,0.517241
more than 60000,0.128098,0.193103


In [40]:
# 종합상품
# 저렴한 차량은 큰차이 없다
# 보험사기 관점에선 비쌀 땐 비율이 더 높음. 중간가격은 반대경향
pd.crosstab(all_p.VehiclePrice_num, all_p.FraudFound_P, normalize='columns')

FraudFound_P,0,1
VehiclePrice_num,,
30000 to 59000,0.179635,0.143805
less than 30000,0.638479,0.623894
more than 60000,0.181886,0.232301


- 두 상품 모두 저렴한 차량의 비율이 높았지만, 이는 전체 데이터셋에서도 마찬가지이다.
- 특징적인 점은, 보험사기에 사용되는 차량의 경우 값이 6000 이상인 차량의 비율이 높아진다는 점이다.
    - All perils의 경우, 평균 차량의 비율이 14%로, 3% 낮아지는 대신 비싼 차량의 비율이 23%로 5% 높아진다.
    - Collision의 경우는 전체적으로 차량의 가치가 높아진다. 보험 사기에 사용되는 차량의 경우 저렴한 모델의 비율이 51%로 10%나 줄어들고, 대신 평균 금액 차량이 3%, 그리고 비싼 차량의 비율이 7% 오른다.
- -> 따라서 보험 사기에 사용되는 차량의 경우 값이 많이 나가는 경향이 있다.

그래서? 비싼 차량이 들어오면 일단은 검사를 받아보는게 좋다는건가?
- 우선은 그렇다. 게다가, 이런 차량들은 비싼 차량들이잖아? 그렇기 때문에 보다 더 꼼꼼하게 보험 사기 검사를 진행할 필요가 있다. 그만큼 비용을 아낄 수 있기 때문이다.
- 다양한 가설들을 여러개 조합 하고, 해당 가설들 중 몇 개 이상에 해당하는 보험 사기 검사를 진행.
- 현재는 방향성만 제시할 수 있고 정확도는 조금 떨어지는 가설만 존재함. 앞으로 지속적으로 데이터를 추가 수집하여 더 데이터 분석에 활용 예정
- 그래야 장기적으로 우리들이 더 많은 패턴을 찾는데 도움을 줄 수 있다.
- 일 년에 n개 이하인 경우 탐색하도록

---

과거 사고 이력도 함께 고려해보자
- 각 상품별로, 가격이 높은 애들만 놓고 봤을 때 과연 어떤 패턴을 보이는가?

In [41]:
# 비싼차만 비율 차이가 많이 났기때문에, 비싼 차 위주로 한번 더 체크
all_p_tmp = all_p.loc[all_p['VehiclePrice_num']=='more than 60000']

In [42]:
o, c = statistical_tests_step1(all_p_tmp, 'FraudFound_P', ['PastNumberOfClaims'])

Relevant : PastNumberOfClaims


In [43]:
plot_two_categorical(all_p_tmp, 'FraudFound_P', 'PastNumberOfClaims')

이런 차량들을 갖고 과거에 여러번 보험 사기를 쳤을 가능성이 있다. 하지만 현재로서는 알아내기 힘들다. (충분한 모수가 없음) 따라서 보다 더 명확한 인사이트를 얻기 위해서는 보험 사기를 더 많이 잡고 데이터를 확보할 수 있는 방안을 찾아야 한다.

- 해당 방법은 다음과 같다
    - 우리들이 제공하는 인사이트는 한 두 개의 변수들을 활용하여 구체적이지 못한 인사이트다.
    - 보다 더 깊게 들어가기 위해서는 해당 변수들 뿐만 아니라 다른 변수들도 함께 확인할 수 있어야 하는데, 현재 데이터의 부족으로 그렇지 못하는 상황이다
    - 따라서 현재 제공하는 1차적인 인사이트를 활용하며 보험 사기를 탐지하자.
    - 물론 정확도는 상대적으로 부족할 수도 있다. 하지만 점진적으로 좋아질 예정이기 때문에, 다음에 데이터가 충분히 쌓인 후에는 보다 더 성능이 좋은 인사이트를 제공해줄 수 있을 것이다.